In [2]:
!pip install -U transformers accelerate peft datasets -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 5.0.0.dev0 which is incompatible.
autogluon-multimodal 1.4.0 requires fsspec[http]<=2025.3, but you have fsspec 2025.10.0 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 5.0.0.dev0 which is incompatible.
autogluon-common 1.4.0 requires pyarrow<21.0.0,>=7.0.0, but you have pyarrow 22.0.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch

data = [
    {"question": "The perimeter of a square with side 3 is:", "options": "A. 6 | B. 9 | C. 12 | D. 3", "answer": "C"},
    {"question": "Which statements are true for parallel lines?", "options": "A. They never intersect | B. They coincide | C. Angles at intersection are equal | D. They are perpendicular", "answer": "AB"},
]

ds = Dataset.from_list(data)

def format_example(ex):
    text = (
        "Instruction: Choose the correct answer from the options.\n"
        f"Question: {ex['question']}\n"
        f"Options: {ex['options']}\n"
        f"Answer: {ex['answer']}"
    )
    return {"text": text}

ds = ds.map(format_example, remove_columns=["question", "options", "answer"])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

## Если данные даны в pickle, то вот:

In [ ]:
import pickle
from datasets import Dataset

with open("data.pickle", "rb") as f:
    loaded_data = pickle.load(f)

ds = Dataset.from_list(loaded_data)

def format_example(ex):
    text = (
        "Instruction: Choose the correct answer from the options.\n"
        f"Question: {ex['question']}\n"
        f"Options: {ex['options']}\n"
        f"Answer: {ex['answer']}"
    )
    return {"text": text}

ds = ds.map(format_example, remove_columns=["question", "options", "answer"])
print(ds[0])

In [ ]:

model_name = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="longest", # можно longest, тут смотри уже по памяти
    )

tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=["text"])


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)


In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)
training_args = TrainingArguments(
    output_dir="./qwen3_lora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True, # при t4!!!
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()



In [ ]:
model.save_pretrained("./qwen3_lora_finetuned")
tokenizer.save_pretrained("./qwen3_lora_finetuned")


In [ ]:
prompt = (
    "Instruction: Choose the correct answer from the options.\n"
    "Question: The perimeter of a square with side 3 is:\n"
    "Options: A. 6 | B. 9 | C. 12 | D. 3\n"
    "Answer:"
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
output_ids = model.generate(input_ids, max_new_tokens=10)
answer = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)
print("Predicted answer:", answer)
